In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

data = pd.read_csv('featured_data.csv')
data['date'] = pd.to_datetime(data['date'])

bins = [-np.inf,-3, -2, -1, -0.5, -0.25, 0.25, 0.5, 1, 2, 3, np.inf]
data['category'] = pd.cut(data['change'], bins=bins, labels=False)

vix_data = pd.read_csv('dataset\^VIX.csv')
vix_data = vix_data.rename(columns={"Date": "date"})
vix_data['date'] = pd.to_datetime(vix_data['date']) 
vix_data.columns = ['vix_' + col if col != 'date' else col for col in vix_data.columns]


merged_data = pd.merge(data, vix_data, on='date', how='inner')


train = merged_data[(merged_data['date'] >= pd.Timestamp(year=1990, month=1, day=1)) & 
                    (merged_data['date'] < pd.Timestamp(year=2022, month=1, day=1))]
test = merged_data[merged_data['date'] >= pd.Timestamp(year=2022, month=1, day=1)]


In [2]:
feature_columns = [
    "rsi_7", "rsi_9", "rsi_14", "rsi_21", "rsi_30", "rsi_50", "rsi_10", "rsi_60", 
    "kdjk", "kdjd", "kdjj", 
    "high_5_roc", "low_5_roc", "high_10_roc", "low_10_roc", "high_15_roc", "low_15_roc", "high_30_roc", "low_30_roc", 
    "close_5_roc", "close_10_roc", "close_15_roc",  "close_30_roc", 
    "macd", "macds", "macdh", "dma", 
    "eribull", "eribear", 
]




for dataset in [train, test]:
    for col in feature_columns:
        for shift_day in range(1, 15):
            dataset[f'{col}_prev_{shift_day}'] = dataset[col].shift(shift_day)


    for shift_day in range(1, 31):
        dataset[f'change_prev_{shift_day}'] = dataset['change'].shift(shift_day)


    dataset.dropna(inplace=True)


scaler = StandardScaler()
list_to_add = ['day_of_week', 'vix_Open']

shifted_feature_columns = [f'{col}_prev_{shift_day}' for col in feature_columns for shift_day in range(1, 15)]
shifted_change_columns = [f'change_prev_{shift_day}' for shift_day in range(1, 31)]
all_shifted_columns = shifted_feature_columns + shifted_change_columns + list_to_add

X_train = scaler.fit_transform(train[all_shifted_columns])
X_test = scaler.transform(test[all_shifted_columns])

y_train = train['category']
y_test = test['category']


C:\Users\gjftn\AppData\Local\Temp\ipykernel_28880\1639997313.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset[f'{col}_prev_{shift_day}'] = dataset[col].shift(shift_day)
C:\Users\gjftn\AppData\Local\Temp\ipykernel_28880\1639997313.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset[f'{col}_prev_{shift_day}'] = dataset[col].shift(shift_day)
C:\Users\gjftn\AppData\Local\Temp\ipykernel_28880\1639997313.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a D

In [5]:
from xgboost import XGBClassifier

xgb_model = XGBClassifier()
xgb_model.fit(X_train, y_train)
xgb_predictions = xgb_model.predict(X_test)


c:\Users\gjftn\OneDrive\바탕 화면\Projects\Nasdaq Pred\HackED_Beta\xgboost.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset[f'{col}_prev_{shift_day}'] = dataset[col].shift(shift_day)
c:\Users\gjftn\OneDrive\바탕 화면\Projects\Nasdaq Pred\HackED_Beta\xgboost.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset[f'{col}_prev_{shift_day}'] = dataset[col].shift(shift_day)
c:\Users\gjftn\OneDrive\바탕 화면\Projects\Nasdaq Pred\HackED_Beta\xgboost.py:35: SettingWithCopyWarning: 
A value is trying to be set

ImportError: cannot import name 'XGBRegressor' from partially initialized module 'xgboost' (most likely due to a circular import) (c:\Users\gjftn\OneDrive\바탕 화면\Projects\Nasdaq Pred\HackED_Beta\xgboost.py)

In [ ]:
from lightgbm import LGBMClassifier

lgbm_model = LGBMClassifier()
lgbm_model.fit(X_train, y_train)
lgbm_predictions = lgbm_model.predict(X_test)


In [ ]:
from catboost import CatBoostClassifier

cat_model = CatBoostClassifier(verbose=0)
cat_model.fit(X_train, y_train)
cat_predictions = cat_model.predict(X_test)


In [ ]:
from sklearn.metrics import accuracy_score, f1_score

print("XGBoost Accuracy:", accuracy_score(y_test, xgb_predictions))
print("XGBoost F1 Score:", f1_score(y_test, xgb_predictions, average='macro'))

print("LGBM Accuracy:", accuracy_score(y_test, lgbm_predictions))
print("LGBM F1 Score:", f1_score(y_test, lgbm_predictions, average='macro'))

print("CatBoost Accuracy:", accuracy_score(y_test, cat_predictions))
print("CatBoost F1 Score:", f1_score(y_test, cat_predictions, average='macro')) 


In [ ]:

xgb_results = pd.DataFrame({'True_Values': y_test, 'XGB_Predictions': xgb_predictions})
lgbm_results = pd.DataFrame({'True_Values': y_test, 'LGBM_Predictions': lgbm_predictions})
cat_results = pd.DataFrame({'True_Values': y_test, 'CatBoost_Predictions': cat_predictions})

xgb_results.to_csv('xgb_predictions.csv', index=False)
lgbm_results.to_csv('lgbm_predictions.csv', index=False)
cat_results.to_csv('catboost_predictions.csv', index=False)
